In [1]:
import os
import pandas as pd
import couchdb
import requests
from prophet import Prophet
from datetime import datetime, timedelta

from dotenv import load_dotenv

c:\Users\fadhil\miniconda3\envs\rdvenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
load_dotenv()

SQLITE_PATH = os.getenv("SQLITE_PATH")
COUCHDB_HOST = os.getenv("COUCHDB_HOST")
COUCH_PORT = os.getenv("COUCH_PORT")
COUCHDB_USERNAME = os.getenv("COUCHDB_USERNAME")
COUCHDB_PASSWORD = os.getenv("COUCHDB_PASSWORD")
FREEWEATHER_DB = os.getenv("FREEWEATHER_DB")
OPENMETEO_DB = os.getenv("OPENMETEO_DB")
OPENWEATHER_DB = os.getenv("OPENWEATHER_DB")
AGGREGATE_DB = os.getenv("AGGREGATE_DB")
COUCHDB_URI = f"http://{COUCHDB_USERNAME}:{COUCHDB_PASSWORD}@{COUCHDB_HOST}:{COUCH_PORT}"

server = couchdb.Server(COUCHDB_URI)
db_name_list = [FREEWEATHER_DB, OPENMETEO_DB, OPENWEATHER_DB]

In [3]:
hournowutc = datetime.utcnow() \
            .replace(minute=0, second=0, microsecond=0) \
            .strftime("%Y-%m-%d %H:%M:%S.%f")
hournow = datetime.now().replace(minute=0, second=0, microsecond=0).strftime("%Y-%m-%d %H:%M:%S.%f")

In [4]:
hournowutc

'2025-06-09 13:00:00.000000'

In [5]:
def flatten_dict(d):
        flat = {}
        for k, v in d.items():
            if isinstance(v, dict):
                flat.update(flatten_dict(v))
            elif isinstance(v, list):
                if v and isinstance(v[0], dict):
                    for item in v:
                        flat.update(flatten_dict(item))
                else:
                    flat[k] = v
            else:
                flat[k] = v
        return flat

def is_near_certain_time(dt, time_filter, tolerance=2):
    """
    Cek apakah waktu (datetime object) mendekati kelipatan 15 menit.
    Misal: 00, 15, 30, atau 45, dengan toleransi ±2 menit.
    """
    if time_filter=="hourly":
        filter = [0]
    elif time_filter=="quarter":
        filter = [0, 15, 30, 45]
    else :
        print(f"There's no filter named {time_filter}")

    minutes = dt.minute
    for base in filter:
        if abs(minutes - base) <= tolerance:
            return True
    return False

def filter_certain_time(data, key="created_at",  time_filter="hourly", tolerance=2):
    """
    Menyaring list of dict (data) agar hanya berisi data yang waktunya 
    mendekati kelipatan 15 menit. `key` adalah field waktu.
    """
    result = []
    for item in data:
        try:
            dt = datetime.strptime(item[key], "%Y-%m-%d %H:%M")
            if is_near_certain_time(dt, tolerance=tolerance, time_filter=time_filter):
                result.append(item)
        except Exception as e:
            print(f"Format waktu salah: {item.get(key)}")
    return result

def get_aggregate():
    now = datetime.now() \
            .replace(minute=0, second=0, microsecond=0) \
            .strftime("%Y-%m-%dT%H:%M:%S.%f")
    
    index_def = {
        "index": {"fields": ["timestamp"]},
        "name": "timestamp_index",
        "type": "json"
    }
    index_url = f"{COUCHDB_URI}/{AGGREGATE_DB}/_index"
    _ = requests.post(index_url, json=index_def)

    query = {
        "selector": {
            "timestamp": {"$lte": now}
        },
        "sort": [{"timestamp": "desc"}],
        "limit": 100000
    }

    temp_list = []
    db = server[AGGREGATE_DB]
    try:
        rows = db.find(query)
    except Exception as e:
        print(e)
    
    for row in rows:
        doc = flatten_dict(row)
        temp_list.append(doc)
    
    return temp_list
    
def get_freeweather():
    hournowutc = datetime.utcnow().replace(minute=0, second=0, microsecond=0).strftime("%Y-%m-%d %H:%M:%S.%f")
    index_def = {
    "index": {"fields": ["created_at"]},
    "name": "timestamp_index",
    "type": "json"
    }
    index_url = f"{COUCHDB_URI}/{FREEWEATHER_DB}/_index"
    _ = requests.post(index_url, json=index_def)

    query = {
        "selector": {
            "created_at": {"$lte": hournowutc}
        },
        "sort": [{"created_at": "desc"}],
        "limit": 100000
    }

    temp_list = []
    db = server[FREEWEATHER_DB]
    try:
        rows = db.find(query)
    except Exception as e:
        print(e)

    for row in rows:
        doc = flatten_dict(row)
        temp_list.append(doc)
    return temp_list

def get_openmeteo():
    nowutc = datetime.utcnow().replace(minute=0, second=0, microsecond=0).strftime("%Y-%m-%d %H:%M:%S.%f")
    index_def = {
    "index": {"fields": ["created_at"]},
    "name": "timestamp_index",
    "type": "json"
    }
    index_url = f"{COUCHDB_URI}/{OPENMETEO_DB}/_index"
    _ = requests.post(index_url, json=index_def)

    query = {
        "selector": {
            "created_at": {
                "$lte": nowutc,
                }
        },
        "sort": [{"created_at": "desc"}],
        "limit": 32
    }

    temp_list = []
    db = server[OPENMETEO_DB]
    try:
        rows = db.find(query)
    except Exception as e:
        print(e)

    for row in rows:
        doc = flatten_dict(row)
        temp_list.append(doc)
    return temp_list


In [6]:
res = get_aggregate()

In [7]:
len(res)

1301

In [8]:
import os
SQLITE_PATH="../airflow/dags/data_store/weather.db"

In [9]:
def get_all_data(db_name): 
    db = server[db_name]
    all_docs = db.view('_all_docs', include_docs=True)

    temp_res = []
    for doc in all_docs:
        doc = doc["doc"]
        flattened_doc = flatten_dict(doc)
        temp_res.append(flattened_doc)
    
    return temp_res

In [10]:
temp_res = get_all_data(AGGREGATE_DB)

df = pd.DataFrame(temp_res)
df.drop(columns=["_id", "_rev", "avg_date", "fields", "created_at", "reduce", "language"], inplace=True)
df.drop(index=[1078, 1079], inplace=True)
df.drop_duplicates(inplace=True)

transformed1 = df.groupby(by=["location_name", "latitude", "longitude", "timestamp"]).mean().reset_index()
transformed1 = transformed1.sort_values(by=["timestamp"], ascending=False)
transformed1 = transformed1[transformed1["latitude"]==(-7.98)].reset_index()
transformed1 = transformed1.drop(columns=["index"])
transformed1 = transformed1.rename(columns=lambda col: col[4:] if col.startswith("avg_") else col)
print(transformed1.shape)
transformed1.head(5)

(290, 11)


,location_name,latitude,longitude,timestamp,wind_gust_kmph,pressure,humidity_pct,wind_speed_kmph,cloud_total_pct,feels_like_c,temperature_c
0,Malang,-7.98,112.63,2025-06-09T19:00:00,4.5,1011.0,97.0,3.6,71.0,19.0,19.0
1,Malang,-7.98,112.63,2025-06-09T18:00:00,6.1,1010.0,93.0,3.6,86.0,21.0,21.0
2,Malang,-7.98,112.63,2025-06-09T17:00:00,5.6,1009.0,87.0,4.0,86.0,26.0,23.9
3,Malang,-7.98,112.63,2025-06-09T16:00:00,3.7,1009.0,81.0,3.6,67.0,26.8,24.7
4,Malang,-7.98,112.63,2025-06-09T15:00:00,4.1,1009.0,76.0,3.6,73.0,27.5,25.5


In [11]:
transformed1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   location_name    290 non-null    object 
 1   latitude         290 non-null    float64
 2   longitude        290 non-null    float64
 3   timestamp        290 non-null    object 
 4   wind_gust_kmph   290 non-null    float64
 5   pressure         290 non-null    float64
 6   humidity_pct     290 non-null    float64
 7   wind_speed_kmph  290 non-null    float64
 8   cloud_total_pct  290 non-null    float64
 9   feels_like_c     290 non-null    float64
 10  temperature_c    290 non-null    float64
dtypes: float64(9), object(2)
memory usage: 25.0+ KB


In [12]:
temp_res = get_all_data(FREEWEATHER_DB)

In [13]:
temp_res

[{'_id': '0000c7ad38e667d67288699fed92ab4a5e42cf957a027052ab66c6832d8b4abf',
  '_rev': '1-80fa305ae2dc3fb766a20fe9195658ce',
  'provider': 'weatherapi',
  'lat': -7.98,
  'lon': 112.63,
  'name': 'Malang',
  'tz_id': 'Asia/Jakarta',
  'region': 'East Java',
  'country': 'Indonesia',
  'localtime': '2025-06-06 14:20',
  'localtime_epoch': 1749194401,
  'uv': 4.6,
  'cloud': 79,
  'is_day': 1,
  'temp_c': 29.2,
  'temp_f': 84.6,
  'vis_km': 10.0,
  'gust_kph': 7.9,
  'gust_mph': 4.9,
  'humidity': 51,
  'wind_dir': 'SW',
  'wind_kph': 6.8,
  'wind_mph': 4.3,
  'code': 1063,
  'icon': '//cdn.weatherapi.com/weather/64x64/day/176.png',
  'text': 'Patchy rain nearby',
  'precip_in': 0.0,
  'precip_mm': 0.01,
  'vis_miles': 6.0,
  'dewpoint_c': 17.9,
  'dewpoint_f': 64.3,
  'feelslike_c': 30.4,
  'feelslike_f': 86.7,
  'heatindex_c': 30.4,
  'heatindex_f': 86.7,
  'pressure_in': 29.85,
  'pressure_mb': 1011.0,
  'wind_degree': 227,
  'windchill_c': 29.2,
  'windchill_f': 84.6,
  'last_updated

In [15]:
def round_down_to_nearest_15(t):
    dt = datetime.strptime(t, "%Y-%m-%d %H:%M")
    minute = (dt.minute // 15) * 15
    dt_rounded = dt.replace(minute=minute, second=0)
    return dt_rounded.strftime("%Y-%m-%d %H:%M")

In [17]:
filtered_data = filter_certain_time(temp_res, key="localtime", tolerance=2, time_filter="quarter")
df = pd.DataFrame(filtered_data)
df.drop(columns=["_id", "_rev", "created_at", "tz_id", "last_updated", "last_updated_epoch", "localtime_epoch"], inplace=True)
df.drop(index=[1078, 1079], inplace=True)
df.drop_duplicates(inplace=True)
df = df.rename(columns={"localtime":"timestamp"})
df["timestamp"] = df["timestamp"].apply(round_down_to_nearest_15)

df = df.sort_values(by=["timestamp"], ascending=False, ignore_index=True).reset_index()
df.drop(columns=["index"], inplace=True)
print(df.shape)

Format waktu salah: None
Format waktu salah: None
(1635, 36)


In [18]:
df["icon"][0]

'//cdn.weatherapi.com/weather/64x64/night/248.png'

In [19]:
df.head()

,provider,name,region,country,lat,lon,timestamp,temp_c,temp_f,is_day,...,windchill_f,heatindex_c,heatindex_f,dewpoint_c,dewpoint_f,vis_km,vis_miles,uv,gust_mph,gust_kph
0,weatherapi,Malang,East Java,Indonesia,-7.98,112.63,2025-06-09 20:15,18.7,65.7,0,...,65.7,18.7,65.7,18.4,65.2,0.0,0.0,0.0,6.7,10.8
1,weatherapi,Malang,East Java,Indonesia,-7.98,112.63,2025-06-09 20:15,18.7,65.7,0,...,65.7,18.7,65.7,18.4,65.2,0.0,0.0,0.0,6.7,10.8
2,weatherapi,Malang,East Java,Indonesia,-7.98,112.63,2025-06-09 20:00,18.7,65.7,0,...,65.7,18.7,65.7,18.4,65.2,0.0,0.0,0.0,6.7,10.8
3,weatherapi,Malang,East Java,Indonesia,-7.98,112.63,2025-06-09 20:00,18.7,65.7,0,...,65.7,18.7,65.7,18.4,65.2,0.0,0.0,0.0,6.7,10.8
4,weatherapi,Malang,East Java,Indonesia,-7.98,112.63,2025-06-09 19:45,19.0,66.1,0,...,66.1,19.0,66.1,18.4,65.2,10.0,6.0,0.0,2.8,4.5


In [20]:
import sqlite3
import pandas as pd

In [21]:
conn = sqlite3.connect(SQLITE_PATH)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS weather_data(
    provider TEXT NOT NULL,
    name TEXT NOT NULL,
    region TEXT NOT NULL,
    country TEXT NOT NULL,
    lat REAL NOT NULL,
    lon REAL NOT NULL,
    timestamp TEXT NOT NULL PRIMARY KEY,

    temp_c REAL NOT NULL,
    temp_f REAL NOT NULL,
    is_day INTEGER NOT NULL,
    text TEXT NOT NULL,
    icon TEXT NOT NULL,
    code INTEGER NOT NULL,

    wind_mph REAL NOT NULL,
    wind_kph REAL NOT NULL,
    wind_degree INTEGER NOT NULL,
    wind_dir TEXT NOT NULL,

    pressure_mb REAL NOT NULL,
    pressure_in REAL NOT NULL,
    precip_mm REAL NOT NULL,
    precip_in REAL NOT NULL,

    humidity INTEGER NOT NULL,
    cloud INTEGER NOT NULL,
    feelslike_c REAL NOT NULL,
    feelslike_f REAL NOT NULL,
    windchill_c REAL NOT NULL,
    windchill_f REAL NOT NULL,
    heatindex_c REAL NOT NULL,
    heatindex_f REAL NOT NULL,
    dewpoint_c REAL NOT NULL,
    dewpoint_f REAL NOT NULL,
    vis_km REAL NOT NULL,
    vis_miles REAL NOT NULL,
    uv REAL NOT NULL,
    gust_mph REAL NOT NULL,
    gust_kph REAL NOT NULL
)
""")

df.to_sql("weather_data", conn, if_exists="replace", index=False)

conn.commit()
conn.close()

In [22]:
conn = sqlite3.connect(SQLITE_PATH)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS weather_summary (
    location_name TEXT NOT NULL,
    latitude REAL NOT NULL,
    longitude REAL NOT NULL,
    timestamp TEXT NOT NULL PRIMARY KEY,

    cloud_total_pct REAL NOT NULL,
    wind_speed_kmph REAL NOT NULL,
    pressure REAL NOT NULL,
    humidity_pct REAL NOT NULL,
    temperature_c REAL NOT NULL,
    feels_like_c REAL NOT NULL,
    wind_gust_kmph REAL NOT NULL
)
""")

transformed1.to_sql("weather_summary", conn, if_exists="replace", index=False)
conn.commit()
conn.close()

In [23]:
conn = sqlite3.connect(SQLITE_PATH)
cursor = conn.cursor()

df = pd.read_sql_query("SELECT * FROM weather_data", conn)
results = df.to_dict(orient="list")

In [24]:
df

,provider,name,region,country,lat,lon,timestamp,temp_c,temp_f,is_day,...,windchill_f,heatindex_c,heatindex_f,dewpoint_c,dewpoint_f,vis_km,vis_miles,uv,gust_mph,gust_kph
0,weatherapi,Malang,East Java,Indonesia,-7.98,112.63,2025-06-09 20:15,18.7,65.7,0,...,65.7,18.7,65.7,18.4,65.2,0.0,0.0,0.0,6.7,10.8
1,weatherapi,Malang,East Java,Indonesia,-7.98,112.63,2025-06-09 20:15,18.7,65.7,0,...,65.7,18.7,65.7,18.4,65.2,0.0,0.0,0.0,6.7,10.8
2,weatherapi,Malang,East Java,Indonesia,-7.98,112.63,2025-06-09 20:00,18.7,65.7,0,...,65.7,18.7,65.7,18.4,65.2,0.0,0.0,0.0,6.7,10.8
3,weatherapi,Malang,East Java,Indonesia,-7.98,112.63,2025-06-09 20:00,18.7,65.7,0,...,65.7,18.7,65.7,18.4,65.2,0.0,0.0,0.0,6.7,10.8
4,weatherapi,Malang,East Java,Indonesia,-7.98,112.63,2025-06-09 19:45,19.0,66.1,0,...,66.1,19.0,66.1,18.4,65.2,10.0,6.0,0.0,2.8,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1630,FreeWeatherAPI,Malang,East Java,Indonesia,-7.98,112.63,2025-05-13 10:00,23.6,74.5,1,...,74.6,25.9,78.6,21.9,71.4,10.0,6.0,2.8,6.7,10.8
1631,FreeWeatherAPI,Malang,East Java,Indonesia,-7.98,112.63,2025-05-13 10:00,23.6,74.5,1,...,74.6,25.9,78.6,21.9,71.4,10.0,6.0,2.8,6.7,10.8
1632,FreeWeatherAPI,Malang,East Java,Indonesia,-7.98,112.63,2025-05-13 09:45,23.1,73.5,1,...,73.5,25.3,77.5,21.0,69.8,10.0,6.0,1.9,6.7,10.8
1633,FreeWeatherAPI,Malang,East Java,Indonesia,-7.98,112.63,2025-05-13 09:30,23.1,73.5,1,...,73.5,25.3,77.5,21.0,69.8,10.0,6.0,1.9,6.7,10.8


In [25]:
conn = sqlite3.connect(SQLITE_PATH)
cursor = conn.cursor()
cursor.execute("""
    SELECT timestamp FROM weather_summary
    ORDER BY timestamp DESC
    LIMIT 1
""")
rows = cursor.fetchall()

In [26]:
rows[0][0]

'2025-06-09T19:00:00'

In [27]:
server = couchdb.Server(COUCHDB_URI)
db = server[AGGREGATE_DB]

def fetch_data() -> dict:
    date_now = datetime.now().strftime("%Y-%m-%d")
    query = {
        "selector" :{
            "timestamp" :{
                "$gte" : date_now
            }
        },
        "sort" : [{"timestamp" : "desc"}],
        "limit":1
    }
    temp_list = []
    try:
        rows = db.find(query)
    except Exception as e:
        print(e)

    for row in rows:
        doc = flatten_dict(row)
        temp_list.append(doc)
    
    return temp_list

In [28]:
fetch_data()

[{'_id': '36a567a83fb9f8f61cca62d84a082fd0',
  '_rev': '1-faacc611f9c32d006293857317c69bbf',
  'location_name': 'Malang',
  'latitude': -7.95,
  'longitude': 112.61,
  'timestamp': '2025-06-09T20:00:00',
  'avg_wind_gust_kmph': 13.7,
  'avg_pressure': 1012.2,
  'avg_humidity_pct': 98.0,
  'avg_wind_speed_kmph': 4.6,
  'avg_cloud_total_pct': 95.0,
  'avg_feels_like_c': 24.1,
  'avg_date': None,
  'avg_temperature_c': 20.7}]

In [29]:
slice=8
transformed1["feels_like_c"].to_list()[-slice:]

[25.4,
 19.1,
 19.100000000000005,
 19.600000000000005,
 19.600000000000005,
 26.199999999999996,
 25.899999999999988,
 25.300000000000004]

In [30]:
from datetime import datetime
import pandas as pd

In [31]:
from sqlite import sqliteModel
from prophetModel import ProphetWrapper

db_model = sqliteModel(SQLITE_PATH)
temps = db_model.get_hourly_precipitation()

In [32]:
# Feels Like Sequencial Making
feels_like = db_model.get_all_hourly_feelslike()
hours = feels_like["timestamp"].dt.strftime("%H:%M")
feelslike_historical = feels_like["feels_like_c"].to_list()[-8:]
# Forecast + Concate
train = feels_like.rename(columns={"timestamp":"ds", "feels_like_c":"y"})
feelslike_model = ProphetWrapper()
feelslike_model.fit(train)
predict = feelslike_model.predict(periods=12)
predict_value = predict["yhat"].to_list()
feelslike_final = feelslike_historical + predict_value
hours = hours + predict["ds"].dt.strftime("%H:%M")
print(len(hours))

# Temperature Sequencial Making
temps = db_model.get_all_hourly_temperature()
temp_historical = temps["temperature_c"].to_list()[-8:]
# Forecast + Concate
train = temps.rename(columns={"timestamp":"ds", "temperature_c":"y"})
temp_model = ProphetWrapper()
temp_model.fit(train)
predict = temp_model.predict(periods=12)
predict_value = predict["yhat"].to_list()
temp_final = temp_historical + predict_value

# Humidity Sequencial Making
humidity = db_model.get_all_hourly_humidity()
humidity_historical = humidity["humidity_pct"].to_list()[-8:]
# Forecast + Concate
train = humidity.rename(columns={"timestamp":"ds", "humidity_pct":"y"})
humidity_model = ProphetWrapper()
humidity_model.fit(train)
predict = humidity_model.predict(periods=12)
predict_value = predict["yhat"].to_list()
humidity_final = humidity_historical + predict_value

# Precipitation Sequencial Making
precipitation = db_model.get_hourly_precipitation()
precipitation_historical = precipitation["precip_mm"].to_list()[-8:]
# Forecast + Concate
train = precipitation.rename(columns={"timestamp":"ds", "precip_mm":"y"})
precipitation_model = ProphetWrapper()
precipitation_model.fit(train)
predict = precipitation_model.predict(periods=12)
predict_value = predict["yhat"].to_list()
precipitation_final = precipitation_historical + predict_value

20:47:05 - cmdstanpy - INFO - Chain [1] start processing
20:47:05 - cmdstanpy - INFO - Chain [1] done processing
c:\Users\fadhil\miniconda3\envs\rdvenv\lib\site-packages\prophet\forecaster.py:1872: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(
20:47:05 - cmdstanpy - INFO - Chain [1] start processing
20:47:05 - cmdstanpy - INFO - Chain [1] done processing


302


c:\Users\fadhil\miniconda3\envs\rdvenv\lib\site-packages\prophet\forecaster.py:1872: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(
20:47:06 - cmdstanpy - INFO - Chain [1] start processing
20:47:06 - cmdstanpy - INFO - Chain [1] done processing
c:\Users\fadhil\miniconda3\envs\rdvenv\lib\site-packages\prophet\forecaster.py:1872: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(
20:47:06 - cmdstanpy - INFO - Chain [1] start processing
20:47:06 - cmdstanpy - INFO - Chain [1] done processing
c:\Users\fadhil\miniconda3\envs\rdvenv\lib\site-packages\prophet\forecaster.py:1872: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(


In [33]:
list(predict["ds"].dt.strftime("%H:%M"))

['21:00',
 '22:00',
 '23:00',
 '00:00',
 '01:00',
 '02:00',
 '03:00',
 '04:00',
 '05:00',
 '06:00',
 '07:00',
 '08:00']

In [34]:
hours = feels_like["timestamp"].dt.strftime("%H:%M").to_list()[-8:]
predict_hour_list = predict["ds"].dt.strftime("%H:%M").to_list()
hours = hours + predict_hour_list

In [35]:
def round_down_to_nearest_15(t):
    dt = datetime.strptime(t, "%H:%M")
    minute = (dt.minute // 15) * 15
    return dt.replace(minute=minute, second=0).strftime("%H:%M")

def get_quarterly_data():
    # Feels Like Sequencial Making
    feels_like = db_model.get_all_quarter_feelslike()
    hours = feels_like["timestamp"].dt.strftime("%H:%M").to_list()[-8:]
    feelslike_historical = feels_like["feelslike_c"].to_list()[-8:]
    # Forecast + Concate
    train = feels_like.rename(columns={"timestamp":"ds", "feelslike_c":"y"})
    feelslike_model = ProphetWrapper()
    feelslike_model.fit(train)
    predict = feelslike_model.predict(periods=20)
    predict_value = predict["yhat"].to_list()
    feelslike_final = feelslike_historical + predict_value
    hours = hours + predict["ds"].dt.strftime("%H:%M").to_list()
    print(hours)
    normalized_hours = [round_down_to_nearest_15(t) for t in hours]

    # Temperature Sequencial Making
    temps = db_model.get_all_quarter_temperature()
    temp_historical = temps["temp_c"].to_list()[-8:]
    # Forecast + Concate
    train = temps.rename(columns={"timestamp":"ds", "temp_c":"y"})
    temp_model = ProphetWrapper()
    temp_model.fit(train)
    predict = temp_model.predict(periods=20)
    predict_value = predict["yhat"].to_list()
    temp_final = temp_historical + predict_value

    # Humidity Sequencial Making
    humidity = db_model.get_all_hourly_humidity()
    humidity_historical = humidity["humidity_pct"].to_list()[-8:]
    # Forecast + Concate
    train = humidity.rename(columns={"timestamp":"ds", "humidity_pct":"y"})
    humidity_model = ProphetWrapper()
    humidity_model.fit(train)
    predict = humidity_model.predict(periods=20)
    predict_value = predict["yhat"].to_list()
    humidity_final = humidity_historical + predict_value

    # Precipitation Sequencial Making
    precipitation = db_model.get_hourly_precipitation()
    precipitation_historical = precipitation["precip_mm"].to_list()[-8:]
    # Forecast + Concate
    train = precipitation.rename(columns={"timestamp":"ds", "precip_mm":"y"})
    precipitation_model = ProphetWrapper()
    precipitation_model.fit(train)
    predict = precipitation_model.predict(periods=20)
    predict_value = predict["yhat"].to_list()
    precipitation_final = precipitation_historical + predict_value

    return pd.DataFrame({
        'Hour': normalized_hours,
        'Temperature': temp_final,
        'Humidity': humidity_final,
        'Feels_Like': feelslike_final,
        "Precipitation" : precipitation_final
    })

In [36]:
data = get_quarterly_data()

20:47:06 - cmdstanpy - INFO - Chain [1] start processing
20:47:06 - cmdstanpy - INFO - Chain [1] done processing
c:\Users\fadhil\miniconda3\envs\rdvenv\lib\site-packages\prophet\forecaster.py:1872: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(
20:47:07 - cmdstanpy - INFO - Chain [1] start processing


['19:15', '19:30', '19:45', '19:45', '20:00', '20:00', '20:15', '20:15', '21:15', '22:15', '23:15', '00:15', '01:15', '02:15', '03:15', '04:15', '05:15', '06:15', '07:15', '08:15', '09:15', '10:15', '11:15', '12:15', '13:15', '14:15', '15:15', '16:15']


20:47:07 - cmdstanpy - INFO - Chain [1] done processing
c:\Users\fadhil\miniconda3\envs\rdvenv\lib\site-packages\prophet\forecaster.py:1872: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(
20:47:07 - cmdstanpy - INFO - Chain [1] start processing
20:47:07 - cmdstanpy - INFO - Chain [1] done processing
c:\Users\fadhil\miniconda3\envs\rdvenv\lib\site-packages\prophet\forecaster.py:1872: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(
20:47:07 - cmdstanpy - INFO - Chain [1] start processing
20:47:08 - cmdstanpy - INFO - Chain [1] done processing
c:\Users\fadhil\miniconda3\envs\rdvenv\lib\site-packages\prophet\forecaster.py:1872: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(
